In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/Users/dmitry/Desktop/Thesis/topics_ds/models')
sys.path.append('/Users/dmitry/Desktop/Thesis/topics_ds/data')
sys.path.append('/Users/dmitry/Desktop/Thesis/topics_ds/pytorch')

In [118]:
import numpy as np

import torch
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical

In [102]:
from models import models as keras_models
from data import data
from pytorch import torch_models

In [5]:
IMG_LEN = 1024
TXT_LEN = 300
N_CLASSES = 50

### Loading

In [8]:
x_img, x_txt, y = data.get_unpacked_data()

### Splitting

In [9]:
x_img_train, x_img_test, x_txt_train, x_txt_test, y_train, y_test = train_test_split(
    x_img, 
    x_txt, 
    y, 
    test_size=0.2, 
    random_state=42,
    stratify=y
)

x_img_train, x_img_val, x_txt_train, x_txt_val, y_train, y_val = train_test_split(
    x_img_train,
    x_txt_train,
    y_train,
    test_size=0.2,
    random_state=42,
    stratify=y_train
)

### Scaling

In [10]:
img_sscaler = StandardScaler()
img_sscaler.fit(x_img_train)

x_img_train = img_sscaler.transform(x_img_train)
x_img_val = img_sscaler.transform(x_img_val)
x_img_test = img_sscaler.transform(x_img_test)

txt_sscaler = StandardScaler()
txt_sscaler.fit(x_txt_train)

x_txt_train = txt_sscaler.transform(x_txt_train)
x_txt_val = txt_sscaler.transform(x_txt_val)
x_txt_test = txt_sscaler.transform(x_txt_test)

### Converting to tensors

In [13]:
x_img_train_t = torch.tensor(x_img_train)
x_img_val_t = torch.tensor(x_img_val)
x_img_test_t = torch.tensor(x_img_test)

x_txt_train_t = torch.tensor(x_txt_train)
x_txt_val_t = torch.tensor(x_txt_val)
x_txt_test_t = torch.tensor(x_txt_test)

y_train_t = torch.tensor(y_train)
y_val_t = torch.tensor(y_val)
y_test_t = torch.tensor(y_test)

### Converting to datasets

In [108]:
train_ds = TensorDataset(x_img_train_t, x_txt_train_t, y_train_t)
val_ds = TensorDataset(x_img_val_t, x_txt_val_t, y_val_t)
test_ds = TensorDataset(x_img_test_t, x_txt_test_t, y_test_t)

### Creating loaders

In [109]:
BATCH_SIZE = 1024

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE)
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

## Minibatch generating function

In [112]:
def get_minibatch(
    n_tasks, 
    n_classes_per_task, 
    n_samples_per_class, 
    x_img_cur, x_txt_cur, y_cur
):    
    n_samples_per_task = n_classes_per_task * n_samples_per_class
    
    x_img_mb = torch.zeros(n_tasks, n_samples_per_task, IMG_LEN)
    x_txt_mb = torch.zeros(n_tasks, n_samples_per_task, TXT_LEN)
    y_mb = torch.zeros(n_tasks, n_samples_per_task, N_CLASSES)
    
    for task in range(n_tasks):
        classes = np.random.choice(N_CLASSES, n_classes_per_task, replace=False)
        i = 0
        for class_num, cur_class in enumerate(classes):
            class_inds = np.where((y_cur==to_categorical(cur_class, N_CLASSES)).all(axis=1))[0]
            example_inds = np.random.choice(
                class_inds, 
                min(n_samples_per_class, len(class_inds)), 
                replace=False
            ) 
            x_img_mb[task][i : i + len(example_inds)] = torch.tensor(x_img_cur[example_inds])
            x_txt_mb[task][i : i + len(example_inds)] = torch.tensor(x_txt_cur[example_inds])
            y_mb[task][i : i + len(example_inds)] = torch.tensor(y_cur[example_inds])
            i += len(example_inds)
            
    return x_img_mb, x_txt_mb, y_mb 

### Creating learner

In [125]:
norm_model = torch_models.NormModel()
norm_optimizer = optim.Adam(norm_model.parameters(), lr=0.001)
norm_writer = SummaryWriter()

### Fitting

In [ ]:
EPOCHS = 10
for epoch in range(EPOCHS):
    n_tasks = 2500
    x_img_mb, x_txt_mb, y_mb = get_minibatch(
        n_tasks=n_tasks, 
        n_classes_per_task=2, 
        n_samples_per_class=128, 
        x_img_cur=x_img_train, 
        x_txt_cur=x_txt_train, 
        y_cur=y_train
    )
    
    for task in range(n_tasks):
        x_img_cur = x_img_mb[task]
        x_txt_cur = x_txt_mb[task]
        y_cur = y_mb[task]
        
        norm_model.zero_grad()
        output = norm_model(x_img_cur.float(), x_txt_cur.float())
        loss = F.nll_loss(output, torch.argmax(y_cur, dim=1))
        loss.backward()
        norm_optimizer.step()
    
    correct = 0
    total = 0
    with torch.no_grad():
        for x_img, x_txt, y in val_loader:
            output = norm_model(x_img.float(), x_txt.float())
            for idx, i in enumerate(output):
                if torch.argmax(i) == torch.argmax(y, dim=1)[idx]:
                    correct += 1
                total += 1
                
    norm_writer.add_scalar('accuracy/norm/val', correct/total, epoch)